In [1]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
import pandas as pd
import math

In [2]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [3]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [4]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [5]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [6]:
emp_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="employee2", properties=jdbc_properties))

In [7]:
w = Window.partitionBy("dep_id").orderBy(desc("salary"))
w1 = Window.partitionBy("dep_id").orderBy("salary")

emp_df_window = emp_df.withColumn("drank_desc", dense_rank().over(w))\
.withColumn("drank_asc", dense_rank().over(w1))\
.withColumn("row_desc", row_number().over(w))\
.withColumn("row_asc", row_number().over(w1))

In [8]:
emp_df_window.groupBy("dep_id").agg(expr("max(case when drank_asc = 1  and row_asc = 1 then emp_name end) low")\
                                   ,expr("max(case when drank_desc = 1  and row_desc = 1 then emp_name end) high")).show()

+------+----+------+
|dep_id| low|  high|
+------+----+------+
|     1|Siva|Prasad|
|     2| Sai|  Ravi|
+------+----+------+

